# 05 - Regression

###### Here, to build a predictive model able to anticipate possible changes and additions in the dataset (in this case, the finished operations historical) a regression model was developed. The regression model seeks to identify patterns in the current dataset and predict the behaviour of additional informations (related to finished operations).

##### Considered Regression Models:

###### - Linear Regression;
###### - Ridge;
###### - Lasso;
###### - ElasticNet;
###### - Decision Tree;
###### - Random Forest;
###### - Gradient Boosting;
###### - XGBoost;
###### - Support Vector Regressor (SVR)
    
----

#### Loading Packages and Database
----

In [7]:
import plotly.io as pio
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
# import fast_dummies
from sklearn.preprocessing import OneHotEncoder
from scipy import stats
import os
import sys
from pathlib import Path

In [8]:
# Adicionar o diretório 'modules_py' ao sys.path
modules_path = Path("modules_py").resolve()
sys.path.append(str(modules_path))

from data_cleaning import DataCleaning_MyData, DataCleaning_GroupData, DataCleaning_OffCod, DataCleaning_OffCod_NotNA, DataCleaning_frequents, DataCleaning_dummies
from aplicaRegressao import aplicar_modelos_regressao

try:
    # Executar a função principal de limpeza de dados
    my_data = DataCleaning_MyData()
    clean_media = DataCleaning_GroupData()
    OffCod_Agru = DataCleaning_OffCod()
    OffCod_Agru_NotNA = DataCleaning_OffCod_NotNA()
    mais_frequentes = DataCleaning_frequents()
    dummies = DataCleaning_dummies()
    
    print("Dados processados com sucesso!")
    display(my_data)
    

# Mostrar o erro caso não execute
except Exception as e:
    print(f"Ocorreu um erro: {e}")

Dados processados com sucesso!


,Fonte,ATIV.Tipo,OFFCod,OFFEscopo,OFFClasse,TipodeProduto,PRODUTOS.Codigo,Produto.Nome,ITENS.Classe,ITENS.Codigo,...,SomaDeHorasApontadas,Poder Linear,Poder Z,Capacidade(TSS/D),Cap Normal Linear,Cap Normal Z,Evaporacao(T/H),Evap Normal Linear,Evap Normal Z,SomaDeHorasApontadasUnitario
0,PropostaAjustada,Fabricação,134,PACKAGE 65 T/H,PACKAGE,Caldeira,12,CC,Operação/Projeto,1100.0,...,504.00,12.000000,-100.000000,NaN,NaN,NaN,65.0,0.000000,-1.000000,504.00
1,PropostaAjustada,Fabricação,134,PACKAGE 65 T/H,PACKAGE,Caldeira,12,CC,Operação/Projeto,1100.0,...,204.00,12.000000,-100.000000,NaN,NaN,NaN,65.0,0.000000,-1.000000,204.00
2,PropostaAjustada,Fabricação,134,PACKAGE 65 T/H,PACKAGE,Caldeira,12,CC,Operação/Projeto,1100.0,...,3.00,12.000000,-100.000000,NaN,NaN,NaN,65.0,0.000000,-1.000000,3.00
3,OffHistorico,Fabricação,121120,CALDEIRA COMPACTA DE 30 T/H x 21 Kgf/Cmå_ x SA...,COMPACTA,Caldeira,12,CC,Operação/Projeto,1100.0,...,449.32,1.470588,-154.865415,NaN,NaN,NaN,30.0,0.014706,-1.548654,449.32
4,OffHistorico,Fabricação,121120,CALDEIRA COMPACTA DE 30 T/H x 21 Kgf/Cmå_ x SA...,COMPACTA,Caldeira,12,CC,Operação/Projeto,1100.0,...,366.41,1.470588,-154.865415,NaN,NaN,NaN,30.0,0.014706,-1.548654,366.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2423,OffAnalise,Fabricação,160042,CALDEIRA RECUPERADORA DE CALOR L-5132001,HRSG,Caldeira,16,CRCHRSG,Vedação / Identificação,1172.1,...,1.00,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,1.00
2424,OffAnalise,Reparo,141565,MELHORIA DA CALDEIRA DE RECUPERAÌàÌÄO QUÌMICA,CRQ,Caldeira,14,CRQ,Vedação / Identificação,1175.1,...,30.00,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,30.00
2425,OffAnalise,Reparo,150145,REFORMA DA CALDEIRA MHI GV-5602,MHI OLEO E GAS,Caldeira,15,CQOG,Vedação / Identificação,1213.1,...,3334.00,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,3334.00
2426,OffAnalise,Fabricação,150151,QUEIMADOR SW 28,QUEIMADOR,Caldeira,15,CQOG,Vedação / Identificação,1213.1,...,944.00,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,944.00


In [9]:
# Usando o filtro do OffCod
OffCod = OffCod_Agru[OffCod_Agru["SomaDeHorasApontadasUnitario"].notna()]
OffCod = OffCod[OffCod["SomaDeHorasApontadasUnitario"] != 0]

# Removendo colunas com dados faltantes
OffCod = OffCod.drop(['Poder Linear', 'Poder Z'], axis= 1)

# Lista de departamentos
departamentos_list = ['CONFORMACAO', 'MONTAGEM', 'SOLDA', 'TRACAGEM', 'USINAGEM']

# Dicionário para armazenar os DataFrames
Departamento = {}

# Criando os DataFrames com um loop
for d in departamentos_list:
    Departamento[d] = OffCod[
        (OffCod['TipodeProduto'] == 'Caldeira') &
        (OffCod['Andrea_DEPARTAMENTO.Descricao'] == d)
    ]

    # Exibir o número de registros para cada departamento
    print(f"Departamento: {d}, Registros: {len(Departamento[d])}")

# Opção 1: Criando uma lista de dataframes para cada departamento
# dfs = [
#     Departamento['CONFORMACAO'],
#     Departamento['MONTAGEM'],
#     Departamento['SOLDA'],
#     Departamento['TRACAGEM'],
#     Departamento['USINAGEM']
#     ]

# Opção 2: Utilizando um único dataframe
dfs = OffCod_Agru_NotNA

display(Departamento['CONFORMACAO'])

# teste
# display(Departamento['CONFORMACAO'])

# display(dfs)

OffCod_Agru_NotNA.dtypes

Departamento: CONFORMACAO, Registros: 46
Departamento: MONTAGEM, Registros: 56
Departamento: SOLDA, Registros: 55
Departamento: TRACAGEM, Registros: 53
Departamento: USINAGEM, Registros: 53


,OFFCod,Andrea_DEPARTAMENTO.Descricao,TipodeProduto,Produto.Nome,SomaDeHorasApontadasUnitario
0,66,CONFORMACAO,Caldeira,CRCHRSG,7066.00
5,71,CONFORMACAO,Caldeira,CQOG,13513.50
26,115,CONFORMACAO,Caldeira,CRCHRSG,1823.00
31,117,CONFORMACAO,Caldeira,CRQ,604.00
44,132,CONFORMACAO,Caldeira,CRQ,1353.00
49,133,CONFORMACAO,Caldeira,CQOG,6899.00
54,134,CONFORMACAO,Caldeira,CC,191.00
59,135,CONFORMACAO,Caldeira,CRCHRSG,7066.00
72,138,CONFORMACAO,Caldeira,CC,328.00
77,139,CONFORMACAO,Caldeira,CND,2715.00


OFFCod                           category
Andrea_DEPARTAMENTO.Descricao    category
TipodeProduto                    category
Produto.Nome                     category
Poder Linear                      float64
Poder Z                           float64
SomaDeHorasApontadasUnitario      float64
dtype: object

In [10]:
print(my_data.isna().sum())

Fonte                               0
ATIV.Tipo                           0
OFFCod                              0
OFFEscopo                           0
OFFClasse                           0
TipodeProduto                       0
PRODUTOS.Codigo                     0
Produto.Nome                        0
ITENS.Classe                       13
ITENS.Codigo                        0
ITENS.Descricao                     5
Andrea_DEPARTAMENTO.Descricao       0
QuantidadePedida                    0
SomaDeHorasApontadas                0
Poder Linear                     1174
Poder Z                             0
Capacidade(TSS/D)                1914
Cap Normal Linear                1914
Cap Normal Z                     1914
Evaporacao(T/H)                  1688
Evap Normal Linear               1688
Evap Normal Z                    1688
SomaDeHorasApontadasUnitario        0
dtype: int64


In [11]:
# Manipulando o MyData

# Removendo colunas com dados faltantes

 # Removendo colunas desnecessárias
columns_to_drop = [
    'QuantidadePedida', 'SomaDeHorasApontadas', 'Poder Linear', 'Poder Z',
    'Capacidade(TSS/D)', 'Cap Normal Linear', 'Cap Normal Z',
    'Evaporacao(T/H)', 'Evap Normal Linear', 'Evap Normal Z'
]
my_data_to_regression = my_data.drop(columns=[col for col in columns_to_drop if col in my_data.columns])

my_data_to_regression = my_data_to_regression.dropna()

print(my_data_to_regression.isna().sum())

Fonte                            0
ATIV.Tipo                        0
OFFCod                           0
OFFEscopo                        0
OFFClasse                        0
TipodeProduto                    0
PRODUTOS.Codigo                  0
Produto.Nome                     0
ITENS.Classe                     0
ITENS.Codigo                     0
ITENS.Descricao                  0
Andrea_DEPARTAMENTO.Descricao    0
SomaDeHorasApontadasUnitario     0
dtype: int64


In [ ]:
resultados_onehot = aplicar_modelos_regressao(my_data_to_regression, target_column='SomaDeHorasApontadasUnitario', encoder='onehot')

resultados_label = aplicar_modelos_regressao(my_data_to_regression, target_column='SomaDeHorasApontadasUnitario', encoder='label')

resultados_target = aplicar_modelos_regressao(my_data_to_regression, target_column='SomaDeHorasApontadasUnitario', encoder='target_enc')

resultados_combinados = pd.concat([resultados_onehot, resultados_label, resultados_target], ignore_index=True)

# Exportar resultados para Excel
resultados_combinados.to_excel('resultados_regressao.xlsx', index=False)

# Exibir os resultados
print(resultados_combinados)

Processando dataset 1 com 'label'


C:\Users\pedro\AppData\Roaming\Python\Python39\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] O sistema não pode encontrar o arquivo especificado
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\pedro\AppData\Roaming\Python\Python39\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
  File "c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\lib\subprocess.py", line 505, in run
    with Popen(*popenargs, **kwargs) as process:
  File "c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\lib\subprocess.py", line 951, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Program Files (x86)\Microsoft Visual Studio\Shared\P

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000153 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 442
[LightGBM] [Info] Number of data points in the train set: 1352, number of used features: 12
[LightGBM] [Info] Start training from score 1136.690491
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000098 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 435
[LightGBM] [Info] Number of data points in the train set: 1352, number of used features: 12
[LightGBM] [Info] Start training from score 1196.336628
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000122 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 435
[LightGBM] [Info] Number of data points in the train set: 1352, number of used features: 12
[LightGBM] [Info] Start tra

C:\Users\pedro\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.903e+06, tolerance: 1.397e+06
  model = cd_fast.enet_coordinate_descent(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000092 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 459
[LightGBM] [Info] Number of data points in the train set: 1352, number of used features: 12
[LightGBM] [Info] Start training from score 1136.690491
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000099 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 459
[LightGBM] [Info] Number of data points in the train set: 1352, number of used features: 12
[LightGBM] [Info] Start training from score 1196.336628
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000156 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 461
[LightGBM] [Info] Number of data points in the train se

In [ ]:
print (my_data.isnull().sum())

Fonte                               0
ATIV.Tipo                           0
OFFCod                              0
OFFEscopo                           0
OFFClasse                           0
TipodeProduto                       0
PRODUTOS.Codigo                     0
Produto.Nome                        0
ITENS.Classe                       13
ITENS.Codigo                        0
ITENS.Descricao                     5
Andrea_DEPARTAMENTO.Descricao       0
QuantidadePedida                    0
SomaDeHorasApontadas                0
Poder Linear                     1174
Poder Z                             0
Capacidade(TSS/D)                1914
Cap Normal Linear                1914
Cap Normal Z                     1914
Evaporacao(T/H)                  1688
Evap Normal Linear               1688
Evap Normal Z                    1688
SomaDeHorasApontadasUnitario        0
dtype: int64
